In [1]:
import numpy as np
def getPlayerGames(Id, season = '2021', seasonType = 'Regular Season'):
    from nba_api.stats.endpoints import playergamelog
    career = playergamelog.PlayerGameLog(player_id = Id , season = season, 
                                           season_type_all_star = seasonType)
    return career.get_data_frames()[0]

game_id = getPlayerGames('203076', '2016')["Game_ID"].to_numpy()
game_id

def getDraftPickID(season = '2021', overall_pick = None, top_picks = None):
    from nba_api.stats.endpoints import drafthistory
    lastDraft = drafthistory.DraftHistory(season_year_nullable = season, 
                                          overall_pick_nullable = overall_pick, topx_nullable = top_picks)
    data = lastDraft.get_dict()
    results = data["resultSets"][0]
    player_data = results["rowSet"][0]
    return player_data[0]

def ComputePlayerPIE(Id, season = '2021', seasonType = 'Regular Season'):
    from nba_api.stats.endpoints import boxscoreadvancedv2, playergamelog
    import numpy as np
    import time
    
    player_log = playergamelog.PlayerGameLog(player_id = Id , season = season, 
                                           season_type_all_star = seasonType)
    
    player_log_df = player_log.get_data_frames()[0]
    game_ids = player_log_df["Game_ID"].to_numpy()
    player_season_PIE = []
    team_season_PIE = []
    
    for ID in game_ids:
        time.sleep(1)
        career = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id = ID)
        player_boxscore = career.get_data_frames()[0]
        player_id_array = player_boxscore['PLAYER_ID'].to_numpy()
        player_row = np.where(player_id_array == int(Id))[0][0]
        player_box = player_boxscore.iloc[[player_row]]
        player_game_pie = player_box['PIE'][player_row]
        player_season_PIE.append(player_game_pie)
        
        team_boxscore = career.get_data_frames()[1]
        team_id_array = team_boxscore['TEAM_ID']
        team_id = player_box['TEAM_ID'][player_row]
        team_row = np.where(team_id_array == team_id)[0][0]
        team_box = team_boxscore.iloc[[team_row]]
        team_game_pie = team_box['PIE'][team_row]
        team_season_PIE.append(team_game_pie)
        
    #print(player_season_PIE)
    player_PIE = sum(player_season_PIE) / len(player_season_PIE)
    team_PIE = sum(team_season_PIE) / len(team_season_PIE)
    
    return player_PIE, team_PIE

#player_id = getDraftPickID(season = '2009', overall_pick = '3')
#ComputePlayerPIE(player_id, '2015')


In [8]:
player_id = getDraftPickID(season = '2009', overall_pick = '3')
print(type(player_id))
#ComputePlayerPIE(player_id, '2015')[0]


<class 'int'>


In [7]:
def getPlayerStats(Id,league = 'nba'):
    from nba_api.stats.endpoints import playercareerstats
    career = playercareerstats.PlayerCareerStats(per_mode36 = 'PerGame',player_id=Id)
    if league == 'college':
        #career = playercareerstats._init_(career_totals_college_season)
        return career.season_totals_college_season.get_data_frame()
    else:
        return career.season_totals_regular_season.get_data_frame()
    
#player_id = getDraftPickID(season = '2009', overall_pick = '3')
#getPlayerStats(player_id)

piea = (29+8.7+8.8-19.7+5.3+(.5*0.8)+7.5+1.7+(.5*.6)-2.8-4.6)
pieb = ((106.5+106.4)+(37.7+39)+(20.4+18.6)-(83.5+85.1)-(29.4+24.6)+(31.7+32.8)+(.5*(11.3+11.8))+(22.2+24.6)+(10+9)+(.5*(5.2+4.9))-(21.8+22.5)-(15.9+16.6))
print(piea, pieb, piea/pieb, (piea/pieb)-0.17502439024390246 )

34.6 176.10000000000002 0.1964792731402612 0.021454882896358746
